In [ ]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# In[2]:


df=pd.read_csv("C:/Users/aless/Desktop/Esame ML e BI/Loan_Default.csv")
print(df.info())


# In[3]:


print(df.isnull().sum())


# In[4]:


df.drop(['ID'],axis =1,inplace = True)

for column in df.columns.tolist():
    temp = df[column].unique()
    if len(temp) < 2:
           df =  df.drop(column, axis = 1)
            


# In[5]:


n_unique_values = 10
all_columns = df.columns.to_list()
categorical_columns = []

for i, column in enumerate(all_columns):
    temp = df.loc[:, column].unique()
    if len(temp) < n_unique_values:
        print (f"{i+1}. {column} has {len(temp)} unique values, which are {temp}") 
        categorical_columns.append(column)
del temp


# In[6]:


df["Gender"] = df["Gender"].replace("Sex Not Available", np.nan)


# In[7]:


#filliamo le variabili categoriche con valori null abbastanza alti
#e poi provvediamo ad eliminare le righe che rimangono con valori null 

values = {"rate_of_interest": df['rate_of_interest'].mean(),
          "Interest_rate_spread": df['Interest_rate_spread'].mean(),
          "Upfront_charges" : df['Upfront_charges'].mean(),
           "term" : df['term'].mean(),
          "property_value" : df['property_value'].mean(),
          "income" : df['income'].mean(),
          "LTV" : df['LTV'].mean(),
          "dtir1" : df['dtir1'].mean()
         }


df.fillna(value = values , inplace = True)


# In[8]:


sns.countplot(data=df , x='loan_type')


# In[9]:


#plotting something

fig,ax=plt.subplots()
sns.barplot(data=df,x='Status',y='Gender',hue='loan_type')
#fig.set_size_inches([12,6])
plt.show()


# In[10]:


#fig,ax=plt.subplots()
sns.barplot(data=df,x='Status',y='loan_amount')
plt.show()


# In[11]:


sns.barplot(x="age", y="loan_amount",hue="Status",  data=df)
plt.show()


# In[12]:


sns.barplot(data=df,x='loan_type',y = 'loan_amount',hue='Status')
plt.show()


# In[13]:


sns.relplot(data=df, x="Credit_Score", y="loan_amount",hue ="Status" , row="age", col="Gender")
plt.show()


# In[14]:


sns.relplot(x ="loan_amount", y ="income",data = df)
plt.show()


# In[15]:


sns.relplot(x ="rate_of_interest", y ="income",data = df)
plt.show()


# In[16]:


df.dropna(inplace = True)


y = df['Status']
df.drop(['Status'], axis = 1,inplace = True)
categorical_columns.remove('Status')


# In[17]:


df = pd.get_dummies(df, columns = categorical_columns)
list(categorical_columns)


# In[18]:


print(df.head(5))


# In[19]:


pd.set_option("display.max_columns", df.shape[-1]) 
print(df.head(5))


# In[20]:


df.info()


# In[21]:


from sklearn.preprocessing import MinMaxScaler

x_col = list(df.columns)
type(x_col)

min_max_scaler = MinMaxScaler()
x_scaled = df.copy()

x_scaled[x_col] = min_max_scaler.fit_transform(df[x_col])


print(x_scaled.info())


# In[22]:


from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.feature_selection import mutual_info_classif


#apply SelectKBest class to extract the k best features
bestfeatures1 = SelectKBest(score_func=mutual_info_classif, k=10)


fit1 = bestfeatures1.fit(x_scaled,y)


dfscores1 = pd.DataFrame(fit1.scores_)


dfcolumns = pd.DataFrame(x_scaled.columns)



featureScores1 = pd.concat([dfcolumns,dfscores1],axis=1)

featureScores1.columns = ['Specs','Score'] 


featureScores1.sort_values(by=['Score'], ascending = False, inplace = True)



# In[24]:


print(featureScores1)


# In[25]:


featureScores1.head(10).plot.bar( x = 'Specs',y = 'Score')
plt.show()


# In[26]:


X = SelectKBest(mutual_info_classif, k=10).fit_transform(x_scaled, y)


# In[27]:


from sklearn.model_selection import train_test_split


x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=1,stratify=y)


# In[29]:


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

numeri1 = list(range(0,40))
for_plot1 = []
              

for num1 in numeri1 :  
    lr=LogisticRegression(max_iter = num1)
    lr.fit(x_train, y_train)
    p_test = lr.predict(x_test)
    acc = accuracy_score(y_test, p_test)
    for_plot1.append(acc)
    
for_plot1

df_for_plot1 = pd.DataFrame(for_plot1, columns = ['Accuracy Model'])
df_for_plot1['numero iterazioni'] =numeri1

sns.factorplot(data =df_for_plot1, x = 'numero iterazioni',y = 'Accuracy Model',height=8, aspect=15/8)
              


# In[30]:


#dopo la 15esima iterazione converge all'accuratezza massima


# In[31]:



import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(p_test,y_test)
plt.show()


# In[32]:


from sklearn.metrics import classification_report
report=classification_report(p_test, y_test) 
print(report)


# In[51]:


print(df_for_plot1.sort_values(by=['Accuracy Model'], ascending = False))


# In[69]:


from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.model_selection import GridSearchCV

dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(x_train, y_train)
plot_tree(dtc, filled=True)
plt.show()




#dt_default = DecisionTreeClassifier(random_state=42)
#dt_default.fit(x_train, y_train)


def evaluate_model(dt_classifier):
    print("Train Accuracy :", accuracy_score(y_train, dt_classifier.predict(x_train)))
    print("Train Confusion Matrix:")
    print(confusion_matrix(y_train, dt_classifier.predict(x_train)))
    print("-"*50)
    print("Test Accuracy :", accuracy_score(y_test, dt_classifier.predict(x_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(x_test)))
    
# Create the parameter grid based on the results of random search 
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=dtc, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")


grid_search.fit(x_train, y_train)



    
    
print(evaluate_model(grid_search))


# In[41]:


score_df = pd.DataFrame(grid_search.cv_results_)
print(score_df.head())


# In[59]:


print("Best Parameters are: ",grid_search.best_estimator_)
dt_best = grid_search.best_estimator_
evaluate_model(dt_best)


# In[44]:


plot_tree(dt_best, filled=True)
plt.figure(figsize=(15,15))
plt.show()


# In[46]:


print(classification_report(y_test, dt_best.predict(x_test)))


# In[ ]:


from sklearn.ensemble import RandomForestClassifier


rfc=RandomForestClassifier(random_state=42)

param = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,3,4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param, cv= 2)# 2-fold cross validation
CV_rfc.fit(x_train, y_train)
print("Best Parameters are: ",CV_rfc.best_estimator_)


# In[ ]:


CV_rfc.best_params_

rfc=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')

rfc.fit(x_train, y_train)

pred=rfc.predict(x_test)

print("Accuracy Random Forest: ",accuracy_score(y_test,rfc.predict(x_test)))




# In[ ]:


from sklearn import tree


fig, axes = plt.subplots(nrows = 1,ncols = 5,figsize = (10,2), dpi=900)
for index in range(0, 5):
    tree.plot_tree(rfc.estimators_[index],
                   filled = True,
                   ax = axes[index]);

    axes[index].set_title('Estimator: ' + str(index+1), fontsize = 11)
fig.savefig('rf_5trees.png')


# In[ ]:


classifiers=[('Logistic Regression',lr),('CART',dtc),('Random Forest',rfc)]


for clf_name,clf in classifiers:
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    print(clf_name,accuracy_score(y_test,y_pred))
    
